In [50]:
import torch
from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer, util
from scipy.spatial.distance import cosine

In [28]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [54]:
# Education_expenditure_and_indicators
#column_headers = ["ID", "Education Sectors", "Periods", "Total Expenditure On Education", "Total Government","Total On Education Institutions", "Lump Sum Financing", "Contract Research", "Operating Costs", "Total Subsidies And Tax Benefits", "Education Component Preschool Day Care", "Transport Of Pupils", "Provisions For Disabled Students", "Allowance For School Costs", "Supervising Costs Of Companies", "Student Grants", "Student Loans", "Total Receipts", "EUSubsidies", "Repaid Student Grants And Loans And", "Interest On Student Loans", "Repayments On Student Loans", "Total Expenditure By Households", "Total On Education Institutions", "Tuition Fees Subsidised Institutions", "Tuition Fees Private Institutions", "Parental Contribution School Activities", "Total Other Expenditures On Education", "Education Component Preschool Day Care", "Books Materials Public Transport And", "Wrongfully Received Student Grants And", "Interest On Student Loans", "Repayments On Student Loans", "Total Allowances Received", "Child Care Allowance Education Component", "Provisions For Disabled Students", "Allowances For School Costs", "Scholarships Of Companies", "Student Grants For School Fees Books", "Student Grants For The Cost Of Living", "Student Loans", "Total Companies", "Total On Education Institutes", "Supervising Trainees And Apprentices", "Tuition Fees Private Institutions", "Contract Research", "Total Other Expenditure On Education", "Child Care Allowance Education Component", "Scholarships Of Companies", "Total Subsidies Received", "Transport Of Pupils", "Supervising Costs Of Companies", "Total Foreign Countries", "Total On Education Institutes", "International Funds", "Contract Research", "EUSubsidies", "Total Expenditure On Education", "In Of GDP", "Per Capita", "Total Government Exp On Education", "In Of GDP", "In Of Total Government Expenditure", "Total On Education Institutions", "Total In Of GDP", "Government", "Households Companies Foreign Countries", "Per Student Including RD", "Per Student Excluding RD"]

# Health_expectancy
#column_headers = ["ID", "Sex", "Age At December31", "Type Of Figure", "Periods", "Life Expectancy", "Life Expectancy In Perceived Good Health", "LEWithout Moderate Severe Limitations", "LEWithout Severe Limitations", "LEWithout Light Moderate Severe Lim", "LEWithout Chronic Morbidity", "LEWithout Chr Morb Excl Hypertension", "LEWithout Psychological Complaints", "Life Expectancy Without GALILimitations", "Life Exp Without Severe GALILimitations"]

# Listed_monuments
#column_headers = ["ID", "Type Of Monument", "Regions", "Periods", "Listed Monuments And Historic Buildings"]

# Livestock
#column_headers = ["ID", "Farm Animals", "Periods", "Livestock"]

# Milk_supply_and_dairy_production
#column_headers = ["ID", "Periods", "Volume", "Fat Content", "Protein Content", "Butter", "Cheese", "Total Milk Powder", "Whole Milk Powder", "Skimmed Milk Powder", "Concentrated Milk", "Whey Powder"]

# Mobility
column_headers = ["ID", "Travel Motives", "Population", "Travel Modes", "Margins", "Region Characteristics", "Periods", "Trips", "Distance Travelled", "Time Travelled", "Trips", "Distance Travelled", "Time Travelled"]

# Plant_protection_products
#column_headers = ["ID", "Plant Protection Products", "Periods", "Sales Active Ingredient", "Sales Active Ingredient Indexnumber"]

# Population_dynamics
#column_headers = ["ID", "Periods", "Population At Start Of Selected Period", "Live Births", "Deaths", "Immigration", "Emigration Including Administrative C", "Net Corrections", "Total Population Growth", "Population At End Of Selected Period"]

# Social_security
#column_headers = ["ID", "Periods", "Total Number Of Disablement Benefits", "Benefits WAO", "Benefits Wajong", "Benefits WAZ", "Total Benefits WIA", "Benefits IVA", "Benefits WGA", "Benefits WWNot Seasonally Adjusted", "Benefits WWSeasonally Adjusted", "Benefits IOW", "Total Benefits Income Support", "Benefits Income Support Up To AOWAge", "Benefits Income Support From AOWAge", "Benefits IOAW", "Benefits IOAZ", "Benefits AOW", "Benefits Anw", "Persons Entitled To AKW"]

# Trade_and_industry
#column_headers = ["ID", "Sector Branches SIC2008", "Periods", "Employee", "Employed Person", "Employee", "Employed Person", "Total", "Net Turnover", "Other Revenues", "Total", "Total", "Total", "Initial Stock Of Raw And Auxiliary Mater", "Purchased Raw And Auxiliary Materials", "Closing Stock Raw And Auxiliary Mat", "Total", "Initial Stock Of Commodities", "Purchased Commodities", "Closing Stock Of Commodities", "Payments To Subcontractors", "Purchase Value Not Elsewhere Classified", "Total", "Gross Wages And Salaries", "Total", "Employer Paid Social Insurance Premiums", "Other Social Insurance Costs", "Pension And Early Retirement Premiums", "Other Personnel Costs", "Total", "Costs Of Energy Use", "Housing Costs", "Equipment And Inventory Costs", "Transport Costs", "Sales Costs", "Communication Costs", "Costs Of Other Services", "Other Liabilities", "Depreciation Fixed Assets", "Operating Results", "Net Financial Income","Balance Of Provisions", "Net Extraordinary Income""Pre Tax Results"]

In [ ]:
topics = [{"label": "Demography", "description": "Vital Statistics"},
          {"label": "Topic2", "description": "Description of Topic2"},
          {"label": "Topic3", "description": "Description of Topic3"}]

In [112]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

In [113]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [134]:
# Example sentences
words = "travel modes"
text = "travel modes, population, margins"

In [135]:
marked_words = "[CLS] " + words + " [SEP]"

In [148]:
# Tokenize our sentence with the BERT tokenizer.
tokenized_words = tokenizer.tokenize(marked_words)

In [149]:
# Map the token strings to their vocabulary indeces.
indexed_tokens_words = tokenizer.convert_tokens_to_ids(tokenized_words)

# Display the words with their indeces.
for tup in zip(tokenized_words, indexed_tokens_words):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

[CLS]           101
travel        3,604
modes        11,583
[SEP]           102


In [150]:
# Mark each of the 22 tokens as belonging to sentence "1".
segments_ids_words = [1] * len(tokenized_words)

print (segments_ids_words)

[1, 1, 1, 1]


In [151]:
# Convert inputs to PyTorch tensors
tokens_tensor_words = torch.tensor([indexed_tokens_words])
segments_tensors_words = torch.tensor([segments_ids_words])

In [152]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

In [153]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():

    outputs = model(tokens_tensor_words, segments_tensors_words)

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states_words = outputs[2]

In [154]:
# `token_vecs` is a tensor with shape [22 x 768]
token_vecs_words = hidden_states_words[-2][0]

In [156]:
words_embedding = torch.mean(token_vecs_words, dim=0)

In [137]:
# Add the special tokens.
marked_text = "[CLS] " + text + " [SEP]"

In [138]:
# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(marked_text)

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

[CLS]           101
travel        3,604
modes        11,583
,             1,010
population    2,313
,             1,010
margins      17,034
[SEP]           102


In [139]:
# Mark each of the 22 tokens as belonging to sentence "1".
segments_ids = [1] * len(tokenized_text)

print (segments_ids)

[1, 1, 1, 1, 1, 1, 1, 1]


In [140]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [141]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [142]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():

    outputs = model(tokens_tensor, segments_tensors)

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

In [145]:
# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings = torch.stack(hidden_states, dim=0)

# Remove dimension 1, the "batches".
token_embeddings = torch.squeeze(token_embeddings, dim=1)

# Swap dimensions 0 and 1.
token_embeddings = token_embeddings.permute(1,0,2)

In [146]:
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

Shape is: 8 x 768


In [147]:
for i, token_str in enumerate(tokenized_text):
  print (i, token_str)

0 [CLS]
1 travel
2 modes
3 ,
4 population
5 ,
6 margins
7 [SEP]


In [155]:
for i, token_str in enumerate(tokenized_words):
  print (i, token_str)

0 [CLS]
1 travel
2 modes
3 [SEP]


In [159]:
from scipy.spatial.distance import cosine

In [1]:
diff = 1 - cosine(token_vecs_sum[10], token_vecs_sum[19])

NameError: name 'cosine' is not defined